In [117]:
import random
from random import shuffle
import glob
import numpy as np
import h5py
import cv2
import os
import pandas as pd
import h5py
import os

In [133]:
shuffle_data = True
image_path = './img/data/*.jpg'
addrs = glob.glob(image_path)
addrs = np.string_(addrs)

In [134]:
if shuffle_data:
   # c = list(zip(addrs, labels))
    shuffle(addrs)
    #addrs, labels = zip(*c)

In [135]:
train_addrs = addrs[0:int(0.8*len(addrs))]

dev_addrs = addrs[int(0.8*len(addrs)):int(0.9*len(addrs))]

test_addrs = addrs[int(0.9*len(addrs)):]


In [136]:
len(addrs)
len(test_addrs)
len(train_addrs)
len(dev_addrs)

226

In [137]:
# address to where you want to save the hdf5 file
train_path = "./Output/v2/train/"
test_path = "./Output/v2/test/"
dev_path = "./Output/v2/dev/"
hdf5_path = './Output/v2/hdf5/blooddata-color.hdf5'
img_width = 100
img_height = 100

In [138]:
def save_images_2_destination(dataset, destDir):
    for i in range(len(dataset)):
        im = dataset[i]
        fileName = (os.path.basename(im)).decode('UTF-8')
        #trunk the file name by removing rotations.
        pos = fileName.find('tif') + 3
        fileName = fileName[: pos]
        fileName = destDir + fileName
        img = cv2.imread(im.decode('utf-8')) #load image
        cv2.imwrite(fileName,img) #write image


In [143]:
save_images_2_destination(dev_addrs, dev_path)

In [144]:
save_images_2_destination(test_addrs, test_path)

In [145]:
#len(train_addrs)
save_images_2_destination(train_addrs, train_path)

In [146]:
def createLabels(addrs):
    labels = []
    for addr in addrs:
        if '10_' in addr:
            labels.append(10)
        elif '20_' in addr:
            labels.append(20)
        elif '30_' in addr:
            labels.append(30)
        elif '40_' in addr:
            labels.append(40)
        elif '50_' in addr:
            labels.append(50)
        elif '60_' in addr:
            labels.append(60)
        elif '70_' in addr:
            labels.append(70)
        elif '80_' in addr:
            labels.append(70)
        elif '90_' in addr:
            labels.append(70)
    return labels

#Change addresses to byte object for createLabel function
test_d = './Output/v2/test/*.tif'
train_d = './Output/v2/train/*.tif'
dev_d = './Output/v2/dev/*.tif'

test_dir = glob.glob(test_d)
train_dir = glob.glob(train_d)
dev_dir = glob.glob(dev_d)


In [147]:

#Create Labels for images
test_labels = createLabels(test_dir)
train_labels = createLabels(train_dir)
dev_labels = createLabels(dev_dir)

In [149]:
len(train_labels)

1811

In [151]:
train_shape = (len(train_addrs), img_height, img_width, 3)
test_shape = (len(test_addrs), img_height, img_width, 3)
dev_shape = (len(dev_addrs), img_height, img_width, 3) # New

# open a hdf5 file and create earrays
hdf5_file = h5py.File(hdf5_path, mode='w')
hdf5_file.create_dataset("train_imgs_resized", train_shape, np.int8)
hdf5_file.create_dataset("test_imgs_resized", test_shape, np.int8)
hdf5_file.create_dataset("dev_imgs_resized", dev_shape, np.int8) #new

#hdf5_file.create_dataset("train_mean", train_shape[1:], np.float32)
hdf5_file.create_dataset("train_labels", (len(train_addrs),), np.int8)
hdf5_file["train_labels"][...] = train_labels
hdf5_file.create_dataset("train_addrs", (len(train_addrs),), dtype="S20")
hdf5_file["train_addrs"][...] = train_addrs
hdf5_file.create_dataset("test_addrs", (len(test_addrs),), dtype="S20")
hdf5_file["test_addrs"][...] = test_addrs
hdf5_file.create_dataset("test_labels", (len(test_addrs),), np.int8)
hdf5_file["test_labels"][...] = test_labels
#New 
hdf5_file.create_dataset("dev_addrs", (len(dev_addrs),), dtype="S20")
hdf5_file["dev_addrs"][...] = dev_addrs
hdf5_file.create_dataset("dev_labels", (len(dev_addrs),), np.int8)
hdf5_file["dev_labels"][...] = dev_labels

In [152]:
# a numpy array to save the mean of the images
mean = np.zeros(train_shape[1:], np.float32)
# loop over train addresses
for i in range(len(train_addrs)):
   
    # read an image 
    addr = train_addrs[i]
    #print(str(addr))
    
    # cv2 load images
    img = cv2.imread(addr.decode('utf-8'))
    
    # save the image and calculate the mean so far
    hdf5_file["train_imgs_resized"][i, ...] = img[None]
    mean += img / float(len(train_labels))

# loop over test addresses
for i in range(len(test_addrs)):
   
    # read an image 
    # cv2 load images
    addr = test_addrs[i]
    img = cv2.imread(addr.decode('utf-8'))
    
    # save the image
    hdf5_file["test_imgs_resized"][i, ...] = img[None]

# loop over dev addresses
for i in range(len(dev_addrs)):
   
    # read an image 
    # cv2 load images
    addr = dev_addrs[i]
    img = cv2.imread(addr.decode('utf-8'))
    
    # save the image
    hdf5_file["dev_imgs_resized"][i, ...] = img[None]
# save the mean and close the hdf5 file
#hdf5_file["train_mean"][...] = mean


In [153]:
hdf5_file.close()